In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pywt
import soundfile as sf
import matplotlib.pyplot as plt
import plotly.offline as py
import plotly.graph_objs as go
from scipy.interpolate import interp1d
py.init_notebook_mode(connected = True)

In [ ]:
def get_envelope(input_signal, repeat = 2):
    # Taking the absolute value
    t = np.arange(input_signal.shape[-1])
    
    absolute_signal = abs (input_signal)

    signal = absolute_signal

    for i in range(repeat):
        # Peak detection
        ## Init
        isPeak = np.zeros(signal.shape[-1], dtype=bool)
        last_idx = signal.shape[-1] - 1

        ## Define subfunction
        def find_prev_specimen():
            if sample_idx == 0:
                return 0.
            return signal[sample_idx-1]

        def find_next_specimen():
            if sample_idx == last_idx:
                return 0. 
            return signal[sample_idx+1]

        ## Main
        for sample_idx, sample in enumerate(signal):
            if sample_idx in [0, last_idx]:
                isPeak[sample_idx] = True
                continue
            prev_specimen = find_prev_specimen()
            next_specimen = find_next_specimen()
            if prev_specimen < sample and sample > next_specimen:
                isPeak[sample_idx] = True

        peaks_signal = signal[isPeak]
        peaks_time = t[isPeak]
        
        f = interp1d(peaks_time, peaks_signal ,kind = 'cubic')
        signal = f(t)

#         signal = np.interp(t, peaks_time, peaks_signal)
    envelope = signal
    return envelope

In [ ]:
bpm = 120
ppqn = 48

In [ ]:
audio, sr = sf.read('./assets/wav/church2.wav')

n_samples = len(audio)
endtime = len(audio)/sr
print(endtime)
t = np.linspace(0,endtime,n_samples)

# sum to mono
if audio.ndim > 1:
    audio = np.sum(audio, axis=1) / 2
print('input data')
plt.plot(audio)
plt.show()

In [ ]:
# Continuous Wavelet Transform
wavelet = pywt.ContinuousWavelet('gaus1')
scale = np.arange(1,257)
coefs, freqs = pywt.cwt(audio, scale, wavelet, 1/sr)

In [ ]:
p = coefs[:,::100]
# p = coefs
t = np.linspace( 0, endtime, p.shape[1])
# p = abs(p)
p = p/abs(p).max()


In [ ]:
n_ticks = int(endtime/60 * bpm * ppqn)
ticks = np.linspace(0, endtime, n_ticks); 
midi_amplitude = np.empty(shape=[p.shape[0], n_ticks])
for i, p_row in enumerate(p):
    envelope = get_envelope(p_row,4)
    midi_amplitude[i] = abs(np.interp(ticks, t, envelope))


In [ ]:
amps_2d = midi_amplitude

n_times = amps_2d.shape[1]
n_freqs = amps_2d.shape[0]

times_2d = np.empty(shape = (1, n_times))
times_2d[0,:] = ticks
times_2d = np.repeat(times_2d, n_freqs, axis=0)

freqs_2d = np.empty(shape = (n_freqs, 1))
freqs_2d[:,0] = freqs
freqs_2d = np.repeat(freqs_2d, n_times, axis=1)

x = times_2d.flatten()
y = freqs_2d.flatten()
z = amps_2d.flatten()

points = np.column_stack([x,y,z]);


In [ ]:
## Plot

trace = go.Scatter3d(
    x = points[:,0],
    y = points[:,1],
    z = points[:,2],
    mode = 'markers',
    marker=dict(
        size=2,
        opacity=0.8,
    )
)

data = [trace]

layout = go.Layout(
    title= 'Partial Data',
    hovermode= 'closest',
    height=800,
    margin=dict(
            l=0,
            r=0,
            b=0,
            t=0
        ),
    scene= dict(
        xaxis= dict(
            title= 'Time',
            ticklen= 5,
            gridwidth= 2,
        ),
        yaxis=dict(
            title= 'Frequency',
            type='log',
            ticklen= 5,
            gridwidth= 2,
        ),
        zaxis=dict(
            title= 'Amplitude',
            ticklen= 5,
            gridwidth= 2,
        )
    )
)
fig = go.Figure(data, layout)
py.iplot(fig, filename='partials')

In [ ]:

data = [
    go.Surface(
        x=t,
        y=freqs,
        z=p
    )
]
layout = go.Layout(
#     width=1080,
    height=720,
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    ),
    scene= dict(
        xaxis= dict(
            title= 'Time',
            ticklen= 5,
            gridwidth= 2,
        ),
        yaxis=dict(
            title= 'Frequency',
            type='log',
            ticklen= 5,
            gridwidth= 2,
        ),
        zaxis=dict(
            title= 'Amplitude',
            ticklen= 5,
            gridwidth= 2,
        )
    )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='coefs')

In [ ]:
freqs.shape

In [ ]:
P = p[160, :]

In [ ]:
envelope = get_envelope(P,4)

In [ ]:
n_ticks = int(endtime/60 * bpm * ppqn)
ticks = np.linspace(0, endtime, n_ticks); 
midi_amplitude = np.interp(ticks, t, envelope)

plt.plot(t, P)
plt.plot(ticks, midi_amplitude)
plt.show()

In [ ]:
## Plot
trace0 = go.Scatter(
    name='original',
    x = t_p,
    y = P,
    mode = 'markers+lines',
    marker=dict(
        size=4,
        opacity=0.8,
    ),
    line=dict(
        width=1
    )
)

trace1 = go.Scatter(
    name='original',
    x = t_p,
    y = envelope,
    mode = 'lines',
    marker=dict(
        size=4,
        opacity=0.8,
        color = 255
    ),
    line=dict(
        width=3,
        color = ('rgb(175, 52, 2)')
    )
)

data = [trace0, trace1]

layout = go.Layout(
    title= 'Partial Data',
    hovermode= 'closest',
    height=800,
    margin=dict(
            l=0,
            r=0,
            b=0,
            t=0
        ),
    scene= dict(
        xaxis= dict(
            title= 'Time',
            ticklen= 5,
            gridwidth= 2,
        ),
        yaxis=dict(
            title= 'Frequency',
            type='log',
            ticklen= 5,
            gridwidth= 2,
        ),
        zaxis=dict(
            title= 'Amplitude',
            ticklen= 5,
            gridwidth= 2,
        )
    )
)
fig = go.Figure(data, layout)
py.iplot(fig, filename='partials')

In [ ]:
## Plot

trace1 = go.Scatter(
    x = ticks,
    y = midi_amplitude,
    mode = 'markers+lines',
    marker=dict(
        size=6,
        opacity=0.8,
        color=200
    ),
    line=dict(
        width=2
    )
)

trace2 = go.Scatter(
    x = t,
    y = P,
    mode = 'markers+lines',
    marker=dict(
        size=3,
        opacity=0.3,
    ),
    line=dict(
        width=0.2
    )
)

data = [trace1, trace2]

layout = go.Layout(
    title= 'Partial Data',
    hovermode= 'closest',
    height=800,
    margin=dict(
            l=0,
            r=0,
            b=0,
            t=0
        ),
    scene= dict(
        xaxis= dict(
            title= 'Time',
            ticklen= 5,
            gridwidth= 2,
        ),
        yaxis=dict(
            title= 'Frequency',
            type='log',
            ticklen= 5,
            gridwidth= 2,
        ),
        zaxis=dict(
            title= 'Amplitude',
            ticklen= 5,
            gridwidth= 2,
        )
    )
)
fig = go.Figure(data, layout)
py.iplot(fig, filename='partials')

In [ ]:

# widths = np.arange(1, 129)
# cwt = signal.cwt(data, signal.ricker, widths)
# cwt = cwt[:,::10000]


In [ ]:
# jsonAmps = interp1(t, p',ticks)';
# jsonAmps = horzcat(jsonAmps, zeros(size(jsonAmps,1),1));
# jsonTimecode = horzcat(ticks,ticks(1,end)*2-ticks(1,end-1));